In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import keras
import pickle

ModuleNotFoundError: No module named 'keras'

In [26]:
df=pd.read_csv("C:\\Users\\Purushoth\\Desktop\\Apple\\AAPL(ClosingVal-today).csv")

In [3]:
df.head()

,Date,Open,High,Low,Volume,Close
0,1/2/1998,0.121652,0.145089,0.120536,718110400,0.145089
1,1/5/1998,0.147321,0.147879,0.135603,651873600,0.141741
2,1/6/1998,0.142299,0.178571,0.131696,1812473600,0.169085
3,1/7/1998,0.167969,0.169643,0.154576,1041622400,0.156250
4,1/8/1998,0.155692,0.166295,0.151228,774020800,0.162388


In [4]:
df.isna().sum()

Date      0
Open      0
High      0
Low       0
Volume    0
Close     0
dtype: int64

In [5]:
df.shape

(5845, 6)

In [6]:
train = df[:4676]
test = df[-1169:]

In [7]:
y = df['Close']
del df['Close']
del df['Date']

In [8]:
x = pd.DataFrame(df)  

In [9]:
x.shape

(5845, 4)

In [10]:
xtrain = x[:4676]
xtest = x[-1169:]

ytrain = y[:4676]
ytest = y[-1169:].values

In [11]:
xtrain.shape

(4676, 4)

In [12]:
ytrain.shape

(4676,)

In [13]:
xtest.shape

(1169, 4)

In [14]:
ytest.shape

(1169,)

In [15]:
scaler = MinMaxScaler(feature_range=(0, 1))
xtrain= pd.DataFrame(scaler.fit_transform(xtrain))

In [16]:
xtrain.head()

,0,1,2,3
0,0.000000,0.000000,0.000000,0.091946
1,0.000766,0.000083,0.000460,0.082973
2,0.000616,0.001000,0.000341,0.240187
3,0.001383,0.000733,0.001040,0.135768
4,0.001016,0.000633,0.000938,0.099519


In [17]:
scaler = MinMaxScaler(feature_range=(0, 1))
xtest= pd.DataFrame(scaler.fit_transform(xtest))

In [18]:
xtest.head()

,0,1,2,3
0,0.004579,0.000253,0.004839,0.187239
1,0.006211,0.000590,0.005940,0.159473
2,0.007674,0.004066,0.007884,0.290107
3,0.010323,0.005583,0.010001,0.165720
4,0.011828,0.006784,0.011837,0.148606


### Random Forest

In [19]:
from sklearn.ensemble import RandomForestRegressor

In [20]:
rf = RandomForestRegressor()
rf.fit(xtrain, ytrain)

RandomForestRegressor()

In [1]:
# rf_predictions = rf.predict(xtest)
# rf_predictions
print(xtest)
# print(ytest)

NameError: name 'xtest' is not defined

In [22]:
def mda(ytest:np.ndarray,predicted:np.ndarray):
    """ Mean Directional Accuracy """
    return np.mean((np.sign(ytest[1:]-ytest[:-1])== np.sign(predicted[1:]- predicted[:-1])).astype(int))

In [23]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, rf_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, rf_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, rf_predictions)))
print('Mean Directional Accuracy:', mda(ytest, rf_predictions))

Mean Absolute Error: 49.47015224418306
Mean Squared Error: 2900.422337132169
Root Mean Squared Error: 53.85556923041636
Mean Directional Accuracy: 0.747431506849315


In [30]:
pickle.dump(rf, open("aapl_todayPrediction.pkl", 'wb'))
print('model saved...')

model saved...


### XGBoost

In [34]:
from xgboost import XGBRegressor

In [35]:
xgb = XGBRegressor()
xgb.fit(xtrain,ytrain)

[19:53:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [36]:
xgb_predictions = xgb.predict(xtest)
xgb_predictions

array([ 0.22083563,  0.24266464,  0.3179381 , ..., 27.665646  ,
       27.149076  , 26.569046  ], dtype=float32)

In [37]:
print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, xgb_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, xgb_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, xgb_predictions)))
print('Mean Directional Accuracy:', mda(ytest, xgb_predictions))

Mean Absolute Error: 49.467923392293194
Mean Squared Error: 2901.1484174233533
Root Mean Squared Error: 53.86230980401187
Mean Directional Accuracy: 0.5916095890410958


### SVM

In [38]:
from sklearn.svm import SVR

In [39]:
svm = SVR()
svm.fit(xtrain,ytrain)

SVR()

In [40]:
svm_predictions = svm.predict(xtest)
svm_predictions

array([ 0.36722355,  0.34698646,  0.72998725, ..., 27.23921046,
       26.90519226, 26.33688953])

In [41]:
print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, svm_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, svm_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, svm_predictions)))
print('Mean Directional Accuracy:', mda(ytest, svm_predictions))

Mean Absolute Error: 49.70486535514405
Mean Squared Error: 2951.517475356564
Root Mean Squared Error: 54.327870152957075
Mean Directional Accuracy: 0.6789383561643836


In [44]:
ytest

array([ 26.4475  ,  26.467501,  26.870001, ..., 122.540001, 120.089996,
       120.589996])